In [1]:
import ase.io as aio
from ase.visualize import view
import io2, ase
import imolecule
import numpy as np
from IPython.display import display, HTML
from rdkit import Chem

np.set_printoptions(precision=4,suppress=True)

def html(_fs,size=(200,200)): 
    renders = (imolecule.draw(f, size=size, display_html=False) for f in _fs)
    columns = ('<div class="col-xs-6 col-sm-3">{}</div>'.format(r) for r in renders)
    return HTML('<div class="row">{}</div>'.format("".join(columns)))

def id2file(lbs):
    return [ 'frag_%s.sdf'%lb for lb in lbs ]

def idx(i,nav,nf):
    ib=i*nav; _ie=(i+1)*nav; ie=nf if _ie>nf else _ie
    return ib,ie

def ase2html(objs):
    'Return the html representation the atoms object as string'
    from tempfile import NamedTemporaryFile as ntf
    renders = []
    for obj in objs:
        if isinstance(obj,str):
            atoms = aio.read(f)
        elif isinstance(obj, ase.Atoms):
            atoms = obj
        else:
            raise '#unknown `obj type'
        with ntf('r+', suffix='.html') as fo:
            atoms.write(fo.name, format='html')
            fo.seek(0)
            renders.append( fo.read() )
    columns = ('<div class="col-xs-6 col-sm-3">{}</div>'.format(r) for r in renders)
    return HTML('<div class="row">{}</div>'.format("".join(columns)))       

def permute_aidx(m, new_aidx):
    na = len(m)
    ias0 = np.arange(na)
    na1 = len(new_aidx)
    if na1 < na:
        ias1 = list(ias0[new_aidx])+list(ias0[na1:])
    else:
        ias1 = ias0
    zsu = m.numbers[ias1]
    psu = m.positions[ias1]
    return ase.Atoms(zsu,psu)

In [2]:
import aqml.cheminfo.lo.dm as cld
import os,sys
import numpy.linalg as LA

np.set_printoptions(precision=4,suppress=True)

def normalize(_vs, _signs=None):
    _vsu = []
    for i,vi in enumerate(_vs):
        vo = np.array(vi)/LA.norm(vi)
        if _signs is not None:
            vo *= _signs[i]
        _vsu.append( vo )
    return np.array(_vsu)

def get_hyb_map(vsp1, vsb1):
    idx = []; vals = []
    for i1,vp1 in enumerate(vsp1):
        _vals = []
        for i2,vb1 in enumerate(vsb1):
            #_vals.append( np.abs(np.dot(vp1,vb1)) )
            _vals.append( np.dot(vp1,vb1) )
        seq = np.argsort(_vals) 
        _id = seq[-1]
        #if not (_vals[seq[-1]] > _vals[seq[-2]]):
        #print ' _vals = ', _vals
        idx.append( _id ); vals.append( _vals[_id] )
    return idx #, vals

def get_dm_obj(obj, basis='sto-3g', meth='b3lyp', idx=None, idx2=None, iprt=False):
    spin=0; a=0.; verbose=3
    if isinstance(obj,str):
        assert os.path.exists(obj)
        m = aio.read(obj)
    else:
        m = obj
    zs, coords = m.numbers, m.positions
    #fno = fn[:-4] + '.out'
    obj2 = cld.density_matrix(zs, coords, output=None, basis=basis, meth=meth, \
                         spin=spin, verbose=verbose, iprt=iprt)
    obj2.calc_ca_dm(idx=idx, idx2=idx2)
    return obj2

def get_dm_aa(obj, ia, ja, bst='sto-3g', ihao=True):
    assert bst=='sto-3g'
    zs = obj.zs
    nheav = (np.array(zs)>1).sum()
    aoidxs = []
    nao0 = nheav * 5 
    for i in [ia,ja]:
        if i <= nheav-1:
            aoidxs.append( [i*5, (i+1)*5] )
        else:
            aoidxs.append( [nao0+(i-nheav), nao0+(i-nheav)*1+1] )
    ias1, ias2 = aoidxs
    #print 'ias1=', ias1, ', ias2=',ias2
    if ihao:
        _dm = obj.dm1
    else:
        _dm = obj.dm0
    return _dm[ias1[0]:ias1[1]][:,ias2[0]:ias2[1]].T

/Users/bing/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
import itertools as itl
def get_shuffle(a,b):
    s = range(1,5)
    seq1 = [ [0]+list(si) for si in list(itl.permutations(s)) ]
    seq2 = [ [0]+list(si) for si in list(itl.permutations(s)) ]
    n = len(seq1)
    d = 999.
    for i in range(n):
        for j in range(n):
            i1 = seq1[i]; i2 = seq2[j]
            di = np.mean(np.abs(b[i1][:,i2]-a))
            if di < d:
                d = di
                i1o = i1; i2o = i2
    return i1o,i2o,d

In [4]:
a = {'a':[1,2], 'b':[2,3], 'c':[3,4]}
b = list(a.values())
print( b, np.min(b))
#np.min( b )

[[1, 2], [2, 3], [3, 4]] 1


In [5]:
fs1 = [ fi+'.xyz' for fi in ['c06h14', 'c07h16', 'c08h18','c12h26'] ]
objs1 = []
for f1 in fs1:
    print (' now ', f1)
    objs1 += [ get_dm_obj(f1)  ]

 now  c06h14.xyz


/Users/bing/anaconda3/lib/python3.6/site-packages/pyscf/gto/mole.py:998: UserWarning: Function mol.dumps drops attribute spin because it is not JSON-serializable
  warnings.warn(msg)


converged SCF energy = -234.09627537053
 now  c07h16.xyz
converged SCF energy = -272.91519284307
 now  c08h18.xyz
converged SCF energy = -311.734113968152
 now  c12h26.xyz
converged SCF energy = -467.009800442332


In [7]:
o1,o2,o3,o4 = objs1

In [9]:
# now test on the same C-H
print( get_dm_aa(o1, 0, 6))
print( get_dm_aa(o2, 0, 7))
print( get_dm_aa(o3, 0, 8))
print( get_dm_aa(o4, 0, 12))

[[-0.0382 -0.0839  0.6234 -0.0706 -0.0713]]
[[-0.0382 -0.0839  0.6234 -0.0706 -0.0713]]
[[-0.0382 -0.0839  0.6234 -0.0706 -0.0713]]
[[-0.0379 -0.0829  0.6231 -0.0701 -0.0701]]


In [13]:
print( get_dm_aa(o1, 0, 6))
print( get_dm_aa(o1, 0, 7))
print( get_dm_aa(o1, 0, 8))

[[-0.0382 -0.0839  0.6234 -0.0706 -0.0713]]
[[-0.0379 -0.0829 -0.0701  0.6231 -0.0702]]
[[-0.0382 -0.0839 -0.0713 -0.0706  0.6234]]


In [14]:
def get_ridx(s1,s2):
    ridxs = []
    for i in s1:
        ridxs.append( s2.index(i) )
    return ridxs

In [15]:
dm1 = get_dm_aa(o1, 0, 6)
dm2 = get_dm_aa(o1, 0, 7)
s1 = [0,1,6,7,8]; s2 = [0,1,7,8,6]; i1 = get_ridx(s1,s2)
print( dm1[:][:,i1]-dm2 )

[[-0.0002 -0.001  -0.0011  0.0003 -0.0004]]


In [20]:
import itertools as itl
list(itl.permutations( range(4) ))

[(0, 1, 2, 3),
 (0, 1, 3, 2),
 (0, 2, 1, 3),
 (0, 2, 3, 1),
 (0, 3, 1, 2),
 (0, 3, 2, 1),
 (1, 0, 2, 3),
 (1, 0, 3, 2),
 (1, 2, 0, 3),
 (1, 2, 3, 0),
 (1, 3, 0, 2),
 (1, 3, 2, 0),
 (2, 0, 1, 3),
 (2, 0, 3, 1),
 (2, 1, 0, 3),
 (2, 1, 3, 0),
 (2, 3, 0, 1),
 (2, 3, 1, 0),
 (3, 0, 1, 2),
 (3, 0, 2, 1),
 (3, 1, 0, 2),
 (3, 1, 2, 0),
 (3, 2, 0, 1),
 (3, 2, 1, 0)]

In [ ]:
print get_dm_aa(o1, 0, 6)
print get_dm_aa(o1, 0, 7)
print get_dm_aa(o1, 0, 8)

In [ ]:
print get_dm_aa(o4, 0, 14)
print get_dm_aa(o4, 0, 12)
print get_dm_aa(o4, 0, 13)

In [ ]:
dm12_o1 = get_dm_aa(o1, 0, 1)
dm12_o4 = get_dm_aa(o4, 0, 1)
#i1_o1 = [2,7,8,9]; i2_o1 = [1,3,10,11]
#i1_o4 = [2,15,13,14]; i2_o4 = [1,3,17,16];
i1 = [0, 1, 4, 2, 3]; i2 = [0, 1, 2, 4, 3]
print dm12_o4[i2][:,i1] - dm12_o1

In [ ]:
dm_H1_o1 = get_dm_aa(o1, 0, 6)[0]
dm_H2_o1 = get_dm_aa(o1, 0, 7)[0]
dm_H3_o1 = get_dm_aa(o1, 0, 8)[0]
dm_H1_o4 = get_dm_aa(o4, 0, 14)[0]
dm_H2_o4 = get_dm_aa(o4, 0, 12)[0]
dm_H3_o4 = get_dm_aa(o4, 0, 13)[0]
print dm_H1_o4[i1] - dm_H1_o1
print dm_H2_o4[i1] - dm_H2_o1
print dm_H3_o4[i1] - dm_H3_o1

In [ ]:
dm1 = get_dm_aa(o1, 0, 3); #print dm1
dm2 = get_dm_aa(o4, 0, 3); #print dm2
i1=[0,1,4,2,3]; i2=[0,1,2,3,4]
print dm1
print dm2[i2][:,i1] - dm1

In [ ]:
dm1 = get_dm_aa(o1, 0, 4); #print dm1
dm2 = get_dm_aa(o4, 0, 4); #print dm2
i1=[0,1,4,2,3]; i2=[0,1,2,3,4]
print dm1
print dm2[i2][:,i1] - dm1

In [ ]:
dm1 = get_dm_aa(o1, 0, 6); #print dm1
dm2 = get_dm_aa(o4, 1, 15); #print dm2
i1 = [0,2,3,1,4]
print dm2[0][i1]
print dm1

In [ ]:
dm1 = get_dm_aa(o4, 0, 2)
dm2 = get_dm_aa(o1, 1, 3) 
i1 = [0,2,1,3,4]; i2 = [0,1,2,3,4]
ddm = dm2[i2][:,i1] - dm1
print ddm, np.mean(np.abs(ddm))

In [ ]:
i1,i2,dd = get_shuffle(dm1,dm2)
print i1,i2,dd